In [6]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [1]:
import sys
sys.path.append("../..") # Adds higher directory to python modules path.
from utilities import aggregate_feature_calculators
from utilities import aggregate_feature_calculators_setting as aggcal
from utilities.parallel import Parallel
from utilities.dfdb import DFDB

from utilities.process.pqueue import *
from utilities.process.pnode import *
from utilities.process.putilities import *

In [2]:
import numpy as np
import pandas as pd
import os
import time
import datetime
import json
import copy
import gc
import warnings
from tqdm import tqdm_notebook, tqdm

import optuna

import lightgbm as lgb
import xgboost as xgb

from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold,TimeSeriesSplit, GroupKFold
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR, SVR
from sklearn.metrics import mean_absolute_error
from sklearn import preprocessing

import eli5
from eli5.sklearn import PermutationImportance

import networkx as nx

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
csv_file_folder =  '../../data/input'
os.listdir(csv_file_folder)

['test.csv',
 'structures',
 'sample_submission.csv',
 'magnetic_shielding_tensors.csv',
 'mulliken_charges.csv',
 'potential_energy.csv',
 'scalar_coupling_contributions.csv',
 'dipole_moments.csv',
 'structures.csv',
 'train.csv']

In [4]:
file_folder =  '../../data/feature'
[f for f in os.listdir(file_folder) if (f.endswith('.pkl')) and (not f.startswith('.'))]

['giba-r-data-table-simple-features-1-17-lb_train.pkl',
 'eem_train.pkl',
 'coulomb-interaction-speed-up_train.pkl',
 'dataset-with-number-of-bonds-between-atoms_test.pkl',
 'keras-neural-net-for-champs_train.pkl',
 'brute-force-feature-engineering_test.pkl',
 'angles-and-distances_test.pkl',
 'giba-r-data-table-simple-features-0-991-lb_test.pkl',
 'keras-neural-net-for-champs_test.pkl',
 'brute-force-feature-engineering-mini_train.pkl',
 'laplaction_matrix_test.pkl',
 'dataset-with-number-of-bonds-between-atoms_train.pkl',
 'coulomb-interaction-speed-up_test.pkl',
 'bonds-from-structure-data_train.pkl',
 'laplaction_matrix_train.pkl',
 'giba-r-data-table-simple-features-0-991-lb_train.pkl',
 'eem_test.pkl',
 'brute-force-feature-engineering_train.pkl',
 'molecule-with-openbabel_train.pkl',
 'molecular-properties-eda-and-models_train.pkl',
 'brute-force-feature-engineering-mini_test.pkl',
 'eachtype_train.pkl',
 'molecular-properties-eda-and-models_test.pkl',
 'bonds-from-structure-dat

In [5]:
mytrial = []

In [ ]:
"""


y=dso, type=0
524
{'columns': ['dist_N_0_y', 'tertiary_distance_6', 'tertiary_distance_9', 'dist_H_2_y', 'dist_C_1_y', 'tertiary_distance_11', 'dist_H_3_y', 'dist_O_0_x', 'inv_dist1', 'yukawa_F.x', 'yukawa_N.y', 'dist_O_1_x', 'yukawa_N.x', 'tertiary_distance_10', 'inv_dist0R', 'tertiary_atom_0', 'vander_C.y', 'eem_1', 'vander_O.x', 'yukawa_O.x', 'atom_index_1_cycle_size_mean', 'dist_N_0_x', 'dist_H_4_y', 'tertiary_distance_4', 'dist_N_1_x', 'yukawa_O.y', 'atom_n', 'dist_H_1_y', 'tertiary_distance_13', 'atom_1_bond_lengths_mean', 'yukawa_C.y', 'atom_1_bond_lengths_max', 'yukawa_C.x', 'dist_C_2_y', 'tertiary_atom_2', 'dist_O_0_y', 'vander_H.y', 'tertiary_distance_7', 'dist_O_1_y', 'tertiary_distance_0', 'tertiary_distance_2', 'dist_N_1_y', 'atom_1_bond_lengths_std', 'dist_C_0_y', 'dist_C_4_y', 'vander_O.y', 'dist_C_1_x', 'tertiary_distance_5', 'dist_C_3_y', 'tertiary_distance_1', 'tertiary_distance_8', 'tertiary_angle_2'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
y=dso, type=1
1009
{'columns': ['tertiary_distance_6', 'max_molecule_atom_1_dist_xyz', 'tertiary_distance_9', 'dist_C_1_y', 'dist_C_3_x', 'yukawa_N.x', 'dist_H_3_y', 'inv_dist1', 'yukawa_F.x', 'inv_dist0R', 'vander_C.y', 'tertiary_atom_0', 'eem_1', 'bond_atom', 'vander_O.x', 'molecule_name.1', 'tertiary_distance_4', 'dist_N_1_x', 'mean_molecule_atom_0_dist_xyz', 'yukawa_O.y', 'inv_dist0E', 'dist_H_1_y', 'tertiary_distance_13', 'atom_1_bond_lengths_max', 'yukawa_C.x', 'dist_C_2_y', 'tertiary_angle_3', 'inv_dist0', 'dist_O_0_y', 'vander_H.y', 'coulomb_H.y', 'tertiary_distance_2', 'tertiary_distance_3', 'tertiary_distance_0', 'distance_center1', 'dist_O_1_y', 'dist_H_0_x', 'dist_C_0_y', 'tertiary_angle_0', 'tertiary_distance_5', 'tertiary_distance_1', 'dist_C_3_y', 'molecule_atom_index_1_dist_std_diff', 'tertiary_angle_2', 'dist_N_0_y', 'eem_0', 'dist_H_2_y', 'vander_F.y', 'dist_C_2_x', 'tertiary_atom_14', 'dist_H_4_x', 'tertiary_distance_11', 'mean_molecule_atom_1_dist_xyz', 'dist_O_0_x', 'yukawa_N.y', 'dist_H_1_x', 'tertiary_distance_10', 'yukawa_O.x', 'atom_index_1_cycle_size_mean', 'dist_N_2_x', 'dist_N_0_x', 'distance_center0', 'atom_1_bond_lengths_mean', 'dist_H_0_y', 'yukawa_C.y', 'tertiary_atom_2', 'tertiary_distance_7', 'vander_C.x', 'tertiary_angle_1', 'atom_1_bond_lengths_std', 'dist_C_4_y', 'dist_C_1_x', 'tertiary_angle_4', 'vander_N.x', 'tertiary_distance_8'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
y=dso, type=2
1104
{'columns': ['tertiary_distance_6', 'max_molecule_atom_1_dist_xyz', 'tertiary_distance_9', 'dist_C_1_y', 'dist_C_3_x', 'yukawa_N.x', 'dist_H_3_y', 'inv_dist1', 'yukawa_F.x', 'inv_dist0R', 'vander_C.y', 'tertiary_atom_0', 'eem_1', 'bond_atom', 'vander_O.x', 'molecule_name.1', 'tertiary_distance_4', 'dist_N_1_x', 'mean_molecule_atom_0_dist_xyz', 'yukawa_O.y', 'inv_dist0E', 'dist_H_1_y', 'tertiary_distance_13', 'atom_1_bond_lengths_max', 'yukawa_C.x', 'dist_C_2_y', 'tertiary_angle_3', 'inv_dist0', 'dist_O_0_y', 'vander_H.y', 'coulomb_H.y', 'tertiary_distance_2', 'tertiary_distance_3', 'tertiary_distance_0', 'distance_center1', 'dist_O_1_y', 'dist_H_0_x', 'dist_C_0_y', 'tertiary_angle_0', 'tertiary_distance_5', 'tertiary_distance_1', 'dist_C_3_y', 'molecule_atom_index_1_dist_std_diff', 'tertiary_angle_2', 'dist_N_0_y', 'eem_0', 'dist_H_2_y', 'vander_F.y', 'dist_C_2_x', 'tertiary_atom_14', 'dist_H_4_x', 'tertiary_distance_11', 'mean_molecule_atom_1_dist_xyz', 'dist_O_0_x', 'yukawa_N.y', 'dist_H_1_x', 'tertiary_distance_10', 'yukawa_O.x', 'atom_index_1_cycle_size_mean', 'dist_N_2_x', 'dist_N_0_x', 'distance_center0', 'atom_1_bond_lengths_mean', 'dist_H_0_y', 'yukawa_C.y', 'tertiary_atom_2', 'tertiary_distance_7', 'vander_C.x', 'tertiary_angle_1', 'atom_1_bond_lengths_std', 'dist_C_4_y', 'dist_C_1_x', 'tertiary_angle_4', 'vander_N.x', 'tertiary_distance_8'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
y=dso, type=3
1199
{'columns': ['tertiary_distance_6', 'max_molecule_atom_1_dist_xyz', 'tertiary_distance_9', 'dist_C_1_y', 'dist_C_3_x', 'yukawa_N.x', 'dist_H_3_y', 'inv_dist1', 'yukawa_F.x', 'inv_dist0R', 'vander_C.y', 'tertiary_atom_0', 'eem_1', 'bond_atom', 'vander_O.x', 'molecule_name.1', 'tertiary_distance_4', 'dist_N_1_x', 'mean_molecule_atom_0_dist_xyz', 'yukawa_O.y', 'inv_dist0E', 'dist_H_1_y', 'tertiary_distance_13', 'atom_1_bond_lengths_max', 'yukawa_C.x', 'dist_C_2_y', 'tertiary_angle_3', 'inv_dist0', 'dist_O_0_y', 'vander_H.y', 'coulomb_H.y', 'tertiary_distance_2', 'tertiary_distance_3', 'tertiary_distance_0', 'distance_center1', 'dist_O_1_y', 'dist_H_0_x', 'dist_C_0_y', 'tertiary_angle_0', 'tertiary_distance_5', 'tertiary_distance_1', 'dist_C_3_y', 'molecule_atom_index_1_dist_std_diff', 'tertiary_angle_2', 'dist_N_0_y', 'eem_0', 'dist_H_2_y', 'vander_F.y', 'dist_C_2_x', 'tertiary_atom_14', 'dist_H_4_x', 'tertiary_distance_11', 'mean_molecule_atom_1_dist_xyz', 'dist_O_0_x', 'yukawa_N.y', 'dist_H_1_x', 'tertiary_distance_10', 'yukawa_O.x', 'atom_index_1_cycle_size_mean', 'dist_N_2_x', 'dist_N_0_x', 'distance_center0', 'atom_1_bond_lengths_mean', 'dist_H_0_y', 'yukawa_C.y', 'tertiary_atom_2', 'tertiary_distance_7', 'vander_C.x', 'tertiary_angle_1', 'atom_1_bond_lengths_std', 'dist_C_4_y', 'dist_C_1_x', 'tertiary_angle_4', 'vander_N.x', 'tertiary_distance_8'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
y=dso, type=4
1398
{'columns': ['inv_distPR', 'min_molecule_atom_0_dist_xyz', 'tertiary_distance_6', 'tertiary_distance_9', 'dist_C_1_y', 'tertiary_distance_15', 'distance_farthest_0', 'dist_to_type_mean', 'dist_C_0_x', 'molecule_atom_index_1_dist_mean_div', 'adN1', 'yukawa_F.y', 'dist_H_3_y', 'yukawa_N.x', 'inv_dist0R', 'vander_C.y', 'eem_1', 'bond_atom', 'vander_O.x', 'molecule_name.1', 'tertiary_distance_4', 'cos_c0', 'dist_N_1_x', 'mean_molecule_atom_0_dist_xyz', 'yukawa_O.y', 'dist_H_1_y', 'tertiary_distance_13', 'atom_1_n_bonds', 'atom_1_bond_lengths_max', 'yukawa_C.x', 'dist_C_2_y', 'atom_1_bond_lengths_min', 'tertiary_angle_3', 'dist_O_0_y', 'coulomb_H.y', 'distance_center1', 'tertiary_distance_2', 'dist_O_1_y', 'adC2', 'dist_N_1_y', 'tertiary_distance_3', 'dist_H_0_x', 'dist_to_type_0_mean', 'dist_C_4_x', 'dist_C_0_y', 'cos_c0_c1', 'tertiary_angle_0', 'adC3', 'molecule_couples', 'tertiary_distance_5', 'tertiary_distance_1', 'molecule_atom_index_1_dist_min_diff', 'dist_C_3_y', 'tertiary_angle_2', 'distN0', 'dist_N_0_y', 'molecule_atom_index_0_dist_min_diff', 'dist_H_2_y', 'dist_C_2_x', 'vander_N.y', 'tertiary_distance_11', 'dist_O_0_x', 'cos_c1', 'yukawa_N.y', 'dist_H_1_x', 'tertiary_distance_10', 'cos_f0', 'yukawa_O.x', 'atom_index_1_cycle_size_mean', 'dist_H_4_y', 'molecule_atom_index_0_dist_mean_div', 'yukawa_C.y', 'tertiary_distance_12', 'molecule_atom_index_0_dist_min_div', 'inv_dist1R', 'dist_xyz', 'tertiary_distance_7', 'linkN', 'vander_C.x', 'tertiary_atom_1', 'tertiary_angle_1', 'atom_1_bond_lengths_std', 'dist_C_4_y', 'vander_O.y', 'dist_to_type_std', 'dist_C_1_x', 'tertiary_angle_4', 'dist_C_3_x', 'tertiary_distance_8'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
y=dso, type=5
1507
{'columns': ['inv_distPR', 'min_molecule_atom_0_dist_xyz', 'tertiary_distance_6', 'tertiary_distance_9', 'dist_C_1_y', 'tertiary_distance_15', 'distance_farthest_0', 'dist_to_type_mean', 'dist_C_0_x', 'molecule_atom_index_1_dist_mean_div', 'adN1', 'yukawa_F.y', 'dist_H_3_y', 'yukawa_N.x', 'inv_dist0R', 'vander_C.y', 'eem_1', 'bond_atom', 'vander_O.x', 'molecule_name.1', 'tertiary_distance_4', 'cos_c0', 'dist_N_1_x', 'mean_molecule_atom_0_dist_xyz', 'yukawa_O.y', 'dist_H_1_y', 'tertiary_distance_13', 'atom_1_n_bonds', 'atom_1_bond_lengths_max', 'yukawa_C.x', 'dist_C_2_y', 'atom_1_bond_lengths_min', 'tertiary_angle_3', 'dist_O_0_y', 'coulomb_H.y', 'distance_center1', 'tertiary_distance_2', 'dist_O_1_y', 'adC2', 'dist_N_1_y', 'tertiary_distance_3', 'dist_H_0_x', 'dist_to_type_0_mean', 'dist_C_4_x', 'dist_C_0_y', 'cos_c0_c1', 'tertiary_angle_0', 'adC3', 'molecule_couples', 'tertiary_distance_5', 'tertiary_distance_1', 'molecule_atom_index_1_dist_min_diff', 'dist_C_3_y', 'tertiary_angle_2', 'distN0', 'dist_N_0_y', 'molecule_atom_index_0_dist_min_diff', 'dist_H_2_y', 'dist_C_2_x', 'vander_N.y', 'tertiary_distance_11', 'dist_O_0_x', 'cos_c1', 'yukawa_N.y', 'dist_H_1_x', 'tertiary_distance_10', 'cos_f0', 'yukawa_O.x', 'atom_index_1_cycle_size_mean', 'dist_H_4_y', 'molecule_atom_index_0_dist_mean_div', 'yukawa_C.y', 'tertiary_distance_12', 'molecule_atom_index_0_dist_min_div', 'inv_dist1R', 'dist_xyz', 'tertiary_distance_7', 'linkN', 'vander_C.x', 'tertiary_atom_1', 'tertiary_angle_1', 'atom_1_bond_lengths_std', 'dist_C_4_y', 'vander_O.y', 'dist_to_type_std', 'dist_C_1_x', 'tertiary_angle_4', 'dist_C_3_x', 'tertiary_distance_8'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
y=dso, type=6
179
{'columns': ['tertiary_distance_6', 'dist_C_4_x', 'dist_C_3_x', 'cos_c1', 'yukawa_O.x', 'coulomb_H.y', 'molecule_atom_index_0_dist_min_diff', 'cos_c0', 'vander_O.y', 'dist_O_0_y', 'molecule_name.1', 'inv_distPR', 'tertiary_distance_18', 'dist_H_0_x', 'dist_O_2_y', 'yukawa_N.x', 'dist_to_type_std', 'adC1', 'dist_C_4_y', 'tertiary_angle_3', 'max_molecule_atom_1_dist_xyz', 'tertiary_distance_14', 'cos_center0_center1', 'dist_to_type_0_mean', 'tertiary_distance_11', 'vander_O.x', 'tertiary_distance_3', 'dist_H_0_y', 'dist_C_3_y', 'dist_C_0_y', 'dist_N_0_y', 'dist_N_1_y', 'dist_O_1_x', 'dist_H_4_x', 'dist_C_2_y', 'dist_O_1_y', 'adN1', 'molecule_dist_mean', 'molecule_atom_index_0_dist_mean_div', 'min_molecule_atom_0_dist_xyz', 'dist_N_1_x', 'dist_O_0_x', 'molecule_atom_index_0_dist_min_div', 'dist_C_2_x', 'distance_c1', 'dist_C_1_y', 'yukawa_C.x', 'tertiary_distance_13', 'dist_to_type_1_mean', 'dist_xyz', 'dist_F_1_y', 'tertiary_distance_12', 'tertiary_atom_16', 'vander_C.x', 'dist_N_0_x', 'tertiary_distance_9', 'tertiary_distance_1', 'molecule_atom_1_dist_min_diff', 'tertiary_distance_16', 'yukawa_O.y', 'yukawa_N.y', 'tertiary_atom_17', 'tertiary_atom_15', 'tertiary_distance_7', 'link0', 'tertiary_distance_2', 'tertiary_angle_2', 'tertiary_angle_0', 'atom_1_bond_lengths_mean', 'yukawa_C.y', 'dist_to_type_mean', 'tertiary_atom_18', 'tertiary_angle_1', 'tertiary_distance_5', 'cos_c0_c1', 'tertiary_distance_10', 'tertiary_distance_8', 'dist_C_1_x', 'vander_C.y'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
y=dso, type=7
668
{'columns': ['dist_C_4_x', 'dist_C_3_x', 'cos_c1', 'molecule_atom_index_0_dist_min_diff', 'cos_c0', 'dist_O_0_y', 'molecule_name.1', 'dist_H_1_y', 'dist_H_1_x', 'dist_H_2_x', 'dist_to_type_std', 'adC1', 'atom_0_bond_lengths_max', 'dist_C_4_y', 'tertiary_angle_3', 'max_molecule_atom_0_dist_xyz', 'dist_to_type_0_mean', 'tertiary_distance_3', 'atom_1_bond_lengths_std', 'dist_H_0_y', 'dist_C_3_y', 'dist_C_0_y', 'dist_N_0_y', 'dist_N_1_y', 'dist_C_2_y', 'dist_O_1_y', 'adC3', 'distance_center1', 'dist_O_0_x', 'atom_index_1_cycle_size_mean', 'tertiary_angle_5', 'tertiary_angle_12', 'adC2', 'dist_C_2_x', 'tertiary_distance_15', 'molecule_atom_index_1_dist_mean_diff', 'dist_C_1_y', 'tertiary_distance_13', 'atom_1_n_bonds', 'dist_to_type_1_mean', 'inv_dist1R', 'dist_xyz', 'tertiary_angle_14', 'tertiary_angle_13', 'dist_N_0_x', 'tertiary_distance_1', 'tertiary_angle_6', 'link1', 'tertiary_atom_0', 'molecule_atom_index_1_dist_min_div', 'tertiary_distance_2', 'tertiary_distance_7', 'tertiary_angle_2', 'tertiary_angle_0', 'atom_1_bond_lengths_mean', 'atom_1_bond_lengths_min', 'atom_1_bond_lengths_max', 'cos_f0', 'dist_to_type_mean', 'tertiary_angle_1', 'tertiary_distance_5', 'cos_c0_c1', 'tertiary_angle_15', 'tertiary_angle_4'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}

"""

In [7]:
# 'scalar_coupling_constant', 'fc', 'sd','pso','dso'
for y in ['dso']:
    for t in np.arange(6,8,1):
        
        print(f'y={y}, type={t}')
        df_train=pd.read_pickle(f'{file_folder}/df_train.gzde', compression='gzip')
        df_train['y'] = df_train[y]
        df_train = df_train[df_train['type']==t]
        n_samples = 500000 if df_train.shape[0] > 500000 else df_train.shape[0]
        df_train = df_train.sample(n_samples).reset_index(drop=True)
        df_test = pd.DataFrame()
        

        param = {
            'columns': df_train.columns.drop(['index', 'y','group', 'scalar_coupling_constant', 'fc', 'sd','pso','dso']).tolist(),
            'cv': {
                'cls': 'KFold',
                'init':{
                    'n_splits': 5,
                    'shuffle': True,
                    'random_state': 42,
                },
            },
            'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}},
            'model': {
                'cls': 'lgb.LGBMRegressor',
                'init': {
                    'learning_rate': 0.2833769330240482,
                    'feature_fraction': 0.8818248470204605,
                    'bagging_fraction': 0.8205197060908092,
                    'min_data_in_leaf': 202,
                    'lambda_l1': 0.017039063121824582,
                    'lambda_l2': 0.8318702431636841,
                    'max_bin': 100,
                    'num_leaves': 255,
                    'random_state': 3895,
                    'n_jobs': 16
                },
                'fit': {}
            },
            'metric': 'mean_absolute_error'
        }



        process_queue = PQueue(df_train, df_test, param, mytrial)
        sort_features = SortFeatureSelectTopNProcess(**{'top_n':200})
        select_topn = RFESelectTopNProcess(**{'n_features_remain':20, 'n_features_to_remove':10})
        remove_useless = RFERemoveUselessFeaturesProcess(**{})
        process_queue.insert_node(sort_features)
        process_queue.insert_node(select_topn)
        process_queue.insert_node(remove_useless)

        try:
            result = process_queue.run()
        except Exception as e:
            print(e.__str__())
        print(len(process_queue.trial))
        print(process_queue.param)
    

y=dso, type=6
179
{'columns': ['tertiary_distance_6', 'dist_C_4_x', 'dist_C_3_x', 'cos_c1', 'yukawa_O.x', 'coulomb_H.y', 'molecule_atom_index_0_dist_min_diff', 'cos_c0', 'vander_O.y', 'dist_O_0_y', 'molecule_name.1', 'inv_distPR', 'tertiary_distance_18', 'dist_H_0_x', 'dist_O_2_y', 'yukawa_N.x', 'dist_to_type_std', 'adC1', 'dist_C_4_y', 'tertiary_angle_3', 'max_molecule_atom_1_dist_xyz', 'tertiary_distance_14', 'cos_center0_center1', 'dist_to_type_0_mean', 'tertiary_distance_11', 'vander_O.x', 'tertiary_distance_3', 'dist_H_0_y', 'dist_C_3_y', 'dist_C_0_y', 'dist_N_0_y', 'dist_N_1_y', 'dist_O_1_x', 'dist_H_4_x', 'dist_C_2_y', 'dist_O_1_y', 'adN1', 'molecule_dist_mean', 'molecule_atom_index_0_dist_mean_div', 'min_molecule_atom_0_dist_xyz', 'dist_N_1_x', 'dist_O_0_x', 'molecule_atom_index_0_dist_min_div', 'dist_C_2_x', 'distance_c1', 'dist_C_1_y', 'yukawa_C.x', 'tertiary_distance_13', 'dist_to_type_1_mean', 'dist_xyz', 'dist_F_1_y', 'tertiary_distance_12', 'tertiary_atom_16', 'vander_C.x

In [7]:
df_train=pd.read_pickle(f'{file_folder}/df_train.gzde', compression='gzip')
df_test=pd.read_pickle(f'{file_folder}/df_test.gzde', compression='gzip')
df_train['y'] = df_train['dso']

In [8]:
columns_list = [
{'columns': ['dist_N_0_y', 'tertiary_distance_6', 'tertiary_distance_9', 'dist_H_2_y', 'dist_C_1_y', 'tertiary_distance_11', 'dist_H_3_y', 'dist_O_0_x', 'inv_dist1', 'yukawa_F.x', 'yukawa_N.y', 'dist_O_1_x', 'yukawa_N.x', 'tertiary_distance_10', 'inv_dist0R', 'tertiary_atom_0', 'vander_C.y', 'eem_1', 'vander_O.x', 'yukawa_O.x', 'atom_index_1_cycle_size_mean', 'dist_N_0_x', 'dist_H_4_y', 'tertiary_distance_4', 'dist_N_1_x', 'yukawa_O.y', 'atom_n', 'dist_H_1_y', 'tertiary_distance_13', 'atom_1_bond_lengths_mean', 'yukawa_C.y', 'atom_1_bond_lengths_max', 'yukawa_C.x', 'dist_C_2_y', 'tertiary_atom_2', 'dist_O_0_y', 'vander_H.y', 'tertiary_distance_7', 'dist_O_1_y', 'tertiary_distance_0', 'tertiary_distance_2', 'dist_N_1_y', 'atom_1_bond_lengths_std', 'dist_C_0_y', 'dist_C_4_y', 'vander_O.y', 'dist_C_1_x', 'tertiary_distance_5', 'dist_C_3_y', 'tertiary_distance_1', 'tertiary_distance_8', 'tertiary_angle_2'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['tertiary_distance_6', 'max_molecule_atom_1_dist_xyz', 'tertiary_distance_9', 'dist_C_1_y', 'dist_C_3_x', 'yukawa_N.x', 'dist_H_3_y', 'inv_dist1', 'yukawa_F.x', 'inv_dist0R', 'vander_C.y', 'tertiary_atom_0', 'eem_1', 'bond_atom', 'vander_O.x', 'molecule_name.1', 'tertiary_distance_4', 'dist_N_1_x', 'mean_molecule_atom_0_dist_xyz', 'yukawa_O.y', 'inv_dist0E', 'dist_H_1_y', 'tertiary_distance_13', 'atom_1_bond_lengths_max', 'yukawa_C.x', 'dist_C_2_y', 'tertiary_angle_3', 'inv_dist0', 'dist_O_0_y', 'vander_H.y', 'coulomb_H.y', 'tertiary_distance_2', 'tertiary_distance_3', 'tertiary_distance_0', 'distance_center1', 'dist_O_1_y', 'dist_H_0_x', 'dist_C_0_y', 'tertiary_angle_0', 'tertiary_distance_5', 'tertiary_distance_1', 'dist_C_3_y', 'molecule_atom_index_1_dist_std_diff', 'tertiary_angle_2', 'dist_N_0_y', 'eem_0', 'dist_H_2_y', 'vander_F.y', 'dist_C_2_x', 'tertiary_atom_14', 'dist_H_4_x', 'tertiary_distance_11', 'mean_molecule_atom_1_dist_xyz', 'dist_O_0_x', 'yukawa_N.y', 'dist_H_1_x', 'tertiary_distance_10', 'yukawa_O.x', 'atom_index_1_cycle_size_mean', 'dist_N_2_x', 'dist_N_0_x', 'distance_center0', 'atom_1_bond_lengths_mean', 'dist_H_0_y', 'yukawa_C.y', 'tertiary_atom_2', 'tertiary_distance_7', 'vander_C.x', 'tertiary_angle_1', 'atom_1_bond_lengths_std', 'dist_C_4_y', 'dist_C_1_x', 'tertiary_angle_4', 'vander_N.x', 'tertiary_distance_8'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['tertiary_distance_6', 'max_molecule_atom_1_dist_xyz', 'tertiary_distance_9', 'dist_C_1_y', 'dist_C_3_x', 'yukawa_N.x', 'dist_H_3_y', 'inv_dist1', 'yukawa_F.x', 'inv_dist0R', 'vander_C.y', 'tertiary_atom_0', 'eem_1', 'bond_atom', 'vander_O.x', 'molecule_name.1', 'tertiary_distance_4', 'dist_N_1_x', 'mean_molecule_atom_0_dist_xyz', 'yukawa_O.y', 'inv_dist0E', 'dist_H_1_y', 'tertiary_distance_13', 'atom_1_bond_lengths_max', 'yukawa_C.x', 'dist_C_2_y', 'tertiary_angle_3', 'inv_dist0', 'dist_O_0_y', 'vander_H.y', 'coulomb_H.y', 'tertiary_distance_2', 'tertiary_distance_3', 'tertiary_distance_0', 'distance_center1', 'dist_O_1_y', 'dist_H_0_x', 'dist_C_0_y', 'tertiary_angle_0', 'tertiary_distance_5', 'tertiary_distance_1', 'dist_C_3_y', 'molecule_atom_index_1_dist_std_diff', 'tertiary_angle_2', 'dist_N_0_y', 'eem_0', 'dist_H_2_y', 'vander_F.y', 'dist_C_2_x', 'tertiary_atom_14', 'dist_H_4_x', 'tertiary_distance_11', 'mean_molecule_atom_1_dist_xyz', 'dist_O_0_x', 'yukawa_N.y', 'dist_H_1_x', 'tertiary_distance_10', 'yukawa_O.x', 'atom_index_1_cycle_size_mean', 'dist_N_2_x', 'dist_N_0_x', 'distance_center0', 'atom_1_bond_lengths_mean', 'dist_H_0_y', 'yukawa_C.y', 'tertiary_atom_2', 'tertiary_distance_7', 'vander_C.x', 'tertiary_angle_1', 'atom_1_bond_lengths_std', 'dist_C_4_y', 'dist_C_1_x', 'tertiary_angle_4', 'vander_N.x', 'tertiary_distance_8'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['tertiary_distance_6', 'max_molecule_atom_1_dist_xyz', 'tertiary_distance_9', 'dist_C_1_y', 'dist_C_3_x', 'yukawa_N.x', 'dist_H_3_y', 'inv_dist1', 'yukawa_F.x', 'inv_dist0R', 'vander_C.y', 'tertiary_atom_0', 'eem_1', 'bond_atom', 'vander_O.x', 'molecule_name.1', 'tertiary_distance_4', 'dist_N_1_x', 'mean_molecule_atom_0_dist_xyz', 'yukawa_O.y', 'inv_dist0E', 'dist_H_1_y', 'tertiary_distance_13', 'atom_1_bond_lengths_max', 'yukawa_C.x', 'dist_C_2_y', 'tertiary_angle_3', 'inv_dist0', 'dist_O_0_y', 'vander_H.y', 'coulomb_H.y', 'tertiary_distance_2', 'tertiary_distance_3', 'tertiary_distance_0', 'distance_center1', 'dist_O_1_y', 'dist_H_0_x', 'dist_C_0_y', 'tertiary_angle_0', 'tertiary_distance_5', 'tertiary_distance_1', 'dist_C_3_y', 'molecule_atom_index_1_dist_std_diff', 'tertiary_angle_2', 'dist_N_0_y', 'eem_0', 'dist_H_2_y', 'vander_F.y', 'dist_C_2_x', 'tertiary_atom_14', 'dist_H_4_x', 'tertiary_distance_11', 'mean_molecule_atom_1_dist_xyz', 'dist_O_0_x', 'yukawa_N.y', 'dist_H_1_x', 'tertiary_distance_10', 'yukawa_O.x', 'atom_index_1_cycle_size_mean', 'dist_N_2_x', 'dist_N_0_x', 'distance_center0', 'atom_1_bond_lengths_mean', 'dist_H_0_y', 'yukawa_C.y', 'tertiary_atom_2', 'tertiary_distance_7', 'vander_C.x', 'tertiary_angle_1', 'atom_1_bond_lengths_std', 'dist_C_4_y', 'dist_C_1_x', 'tertiary_angle_4', 'vander_N.x', 'tertiary_distance_8'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['inv_distPR', 'min_molecule_atom_0_dist_xyz', 'tertiary_distance_6', 'tertiary_distance_9', 'dist_C_1_y', 'tertiary_distance_15', 'distance_farthest_0', 'dist_to_type_mean', 'dist_C_0_x', 'molecule_atom_index_1_dist_mean_div', 'adN1', 'yukawa_F.y', 'dist_H_3_y', 'yukawa_N.x', 'inv_dist0R', 'vander_C.y', 'eem_1', 'bond_atom', 'vander_O.x', 'molecule_name.1', 'tertiary_distance_4', 'cos_c0', 'dist_N_1_x', 'mean_molecule_atom_0_dist_xyz', 'yukawa_O.y', 'dist_H_1_y', 'tertiary_distance_13', 'atom_1_n_bonds', 'atom_1_bond_lengths_max', 'yukawa_C.x', 'dist_C_2_y', 'atom_1_bond_lengths_min', 'tertiary_angle_3', 'dist_O_0_y', 'coulomb_H.y', 'distance_center1', 'tertiary_distance_2', 'dist_O_1_y', 'adC2', 'dist_N_1_y', 'tertiary_distance_3', 'dist_H_0_x', 'dist_to_type_0_mean', 'dist_C_4_x', 'dist_C_0_y', 'cos_c0_c1', 'tertiary_angle_0', 'adC3', 'molecule_couples', 'tertiary_distance_5', 'tertiary_distance_1', 'molecule_atom_index_1_dist_min_diff', 'dist_C_3_y', 'tertiary_angle_2', 'distN0', 'dist_N_0_y', 'molecule_atom_index_0_dist_min_diff', 'dist_H_2_y', 'dist_C_2_x', 'vander_N.y', 'tertiary_distance_11', 'dist_O_0_x', 'cos_c1', 'yukawa_N.y', 'dist_H_1_x', 'tertiary_distance_10', 'cos_f0', 'yukawa_O.x', 'atom_index_1_cycle_size_mean', 'dist_H_4_y', 'molecule_atom_index_0_dist_mean_div', 'yukawa_C.y', 'tertiary_distance_12', 'molecule_atom_index_0_dist_min_div', 'inv_dist1R', 'dist_xyz', 'tertiary_distance_7', 'linkN', 'vander_C.x', 'tertiary_atom_1', 'tertiary_angle_1', 'atom_1_bond_lengths_std', 'dist_C_4_y', 'vander_O.y', 'dist_to_type_std', 'dist_C_1_x', 'tertiary_angle_4', 'dist_C_3_x', 'tertiary_distance_8'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['inv_distPR', 'min_molecule_atom_0_dist_xyz', 'tertiary_distance_6', 'tertiary_distance_9', 'dist_C_1_y', 'tertiary_distance_15', 'distance_farthest_0', 'dist_to_type_mean', 'dist_C_0_x', 'molecule_atom_index_1_dist_mean_div', 'adN1', 'yukawa_F.y', 'dist_H_3_y', 'yukawa_N.x', 'inv_dist0R', 'vander_C.y', 'eem_1', 'bond_atom', 'vander_O.x', 'molecule_name.1', 'tertiary_distance_4', 'cos_c0', 'dist_N_1_x', 'mean_molecule_atom_0_dist_xyz', 'yukawa_O.y', 'dist_H_1_y', 'tertiary_distance_13', 'atom_1_n_bonds', 'atom_1_bond_lengths_max', 'yukawa_C.x', 'dist_C_2_y', 'atom_1_bond_lengths_min', 'tertiary_angle_3', 'dist_O_0_y', 'coulomb_H.y', 'distance_center1', 'tertiary_distance_2', 'dist_O_1_y', 'adC2', 'dist_N_1_y', 'tertiary_distance_3', 'dist_H_0_x', 'dist_to_type_0_mean', 'dist_C_4_x', 'dist_C_0_y', 'cos_c0_c1', 'tertiary_angle_0', 'adC3', 'molecule_couples', 'tertiary_distance_5', 'tertiary_distance_1', 'molecule_atom_index_1_dist_min_diff', 'dist_C_3_y', 'tertiary_angle_2', 'distN0', 'dist_N_0_y', 'molecule_atom_index_0_dist_min_diff', 'dist_H_2_y', 'dist_C_2_x', 'vander_N.y', 'tertiary_distance_11', 'dist_O_0_x', 'cos_c1', 'yukawa_N.y', 'dist_H_1_x', 'tertiary_distance_10', 'cos_f0', 'yukawa_O.x', 'atom_index_1_cycle_size_mean', 'dist_H_4_y', 'molecule_atom_index_0_dist_mean_div', 'yukawa_C.y', 'tertiary_distance_12', 'molecule_atom_index_0_dist_min_div', 'inv_dist1R', 'dist_xyz', 'tertiary_distance_7', 'linkN', 'vander_C.x', 'tertiary_atom_1', 'tertiary_angle_1', 'atom_1_bond_lengths_std', 'dist_C_4_y', 'vander_O.y', 'dist_to_type_std', 'dist_C_1_x', 'tertiary_angle_4', 'dist_C_3_x', 'tertiary_distance_8'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['tertiary_distance_6', 'dist_C_4_x', 'dist_C_3_x', 'cos_c1', 'yukawa_O.x', 'coulomb_H.y', 'molecule_atom_index_0_dist_min_diff', 'cos_c0', 'vander_O.y', 'dist_O_0_y', 'molecule_name.1', 'inv_distPR', 'tertiary_distance_18', 'dist_H_0_x', 'dist_O_2_y', 'yukawa_N.x', 'dist_to_type_std', 'adC1', 'dist_C_4_y', 'tertiary_angle_3', 'max_molecule_atom_1_dist_xyz', 'tertiary_distance_14', 'cos_center0_center1', 'dist_to_type_0_mean', 'tertiary_distance_11', 'vander_O.x', 'tertiary_distance_3', 'dist_H_0_y', 'dist_C_3_y', 'dist_C_0_y', 'dist_N_0_y', 'dist_N_1_y', 'dist_O_1_x', 'dist_H_4_x', 'dist_C_2_y', 'dist_O_1_y', 'adN1', 'molecule_dist_mean', 'molecule_atom_index_0_dist_mean_div', 'min_molecule_atom_0_dist_xyz', 'dist_N_1_x', 'dist_O_0_x', 'molecule_atom_index_0_dist_min_div', 'dist_C_2_x', 'distance_c1', 'dist_C_1_y', 'yukawa_C.x', 'tertiary_distance_13', 'dist_to_type_1_mean', 'dist_xyz', 'dist_F_1_y', 'tertiary_distance_12', 'tertiary_atom_16', 'vander_C.x', 'dist_N_0_x', 'tertiary_distance_9', 'tertiary_distance_1', 'molecule_atom_1_dist_min_diff', 'tertiary_distance_16', 'yukawa_O.y', 'yukawa_N.y', 'tertiary_atom_17', 'tertiary_atom_15', 'tertiary_distance_7', 'link0', 'tertiary_distance_2', 'tertiary_angle_2', 'tertiary_angle_0', 'atom_1_bond_lengths_mean', 'yukawa_C.y', 'dist_to_type_mean', 'tertiary_atom_18', 'tertiary_angle_1', 'tertiary_distance_5', 'cos_c0_c1', 'tertiary_distance_10', 'tertiary_distance_8', 'dist_C_1_x', 'vander_C.y'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['dist_C_4_x', 'dist_C_3_x', 'cos_c1', 'molecule_atom_index_0_dist_min_diff', 'cos_c0', 'dist_O_0_y', 'molecule_name.1', 'dist_H_1_y', 'dist_H_1_x', 'dist_H_2_x', 'dist_to_type_std', 'adC1', 'atom_0_bond_lengths_max', 'dist_C_4_y', 'tertiary_angle_3', 'max_molecule_atom_0_dist_xyz', 'dist_to_type_0_mean', 'tertiary_distance_3', 'atom_1_bond_lengths_std', 'dist_H_0_y', 'dist_C_3_y', 'dist_C_0_y', 'dist_N_0_y', 'dist_N_1_y', 'dist_C_2_y', 'dist_O_1_y', 'adC3', 'distance_center1', 'dist_O_0_x', 'atom_index_1_cycle_size_mean', 'tertiary_angle_5', 'tertiary_angle_12', 'adC2', 'dist_C_2_x', 'tertiary_distance_15', 'molecule_atom_index_1_dist_mean_diff', 'dist_C_1_y', 'tertiary_distance_13', 'atom_1_n_bonds', 'dist_to_type_1_mean', 'inv_dist1R', 'dist_xyz', 'tertiary_angle_14', 'tertiary_angle_13', 'dist_N_0_x', 'tertiary_distance_1', 'tertiary_angle_6', 'link1', 'tertiary_atom_0', 'molecule_atom_index_1_dist_min_div', 'tertiary_distance_2', 'tertiary_distance_7', 'tertiary_angle_2', 'tertiary_angle_0', 'atom_1_bond_lengths_mean', 'atom_1_bond_lengths_min', 'atom_1_bond_lengths_max', 'cos_f0', 'dist_to_type_mean', 'tertiary_angle_1', 'tertiary_distance_5', 'cos_c0_c1', 'tertiary_angle_15', 'tertiary_angle_4'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
]

In [9]:
mytrial=[]
df_his, df_feature_importances, df_valid_pred, df_test_pred = pd.DataFrame(), pd.DataFrame(),pd.DataFrame(),pd.DataFrame()
for t in  df_train.type.unique().tolist():
    df_his_i, df_feature_importances_i, df_valid_pred_i, df_test_pred_i =  sk_process(df_train[df_train['type']==t].reset_index(drop=True), columns_list[t], f'modeling for {t}', df_test=df_test[df_test['type']==t].reset_index(drop=True), trial=mytrial, is_output_feature_importance=False, trial_level=1)
    df_his = pd.concat([df_his, df_his_i], axis=0)
    df_feature_importances = pd.concat([df_feature_importances, df_feature_importances_i], axis=0)
    df_valid_pred = pd.concat([df_valid_pred, df_valid_pred_i], axis=0)
    df_test_pred = pd.concat([df_test_pred, df_test_pred_i], axis=0)

df_valid_pred = df_valid_pred.sort_values(by=['index']).reset_index(drop=True)
df_test_pred = df_test_pred.sort_values(by=['index']).reset_index(drop=True)

In [10]:
df_trial = pd.DataFrame(mytrial)
df_trial[['datetime', 'message', 'nfeatures', 'train_metric_mean', 'val_metric_mean', 'trn_val_metric_diff', 'message']]

,datetime,message,nfeatures,train_metric_mean,val_metric_mean,trn_val_metric_diff,message
0,2019-07-25 06:34:40.609884,modeling for 0,52,0.005811,0.006486,0.000675,modeling for 0
1,2019-07-25 06:35:05.389278,modeling for 3,75,0.015355,0.019028,0.003672,modeling for 3
2,2019-07-25 06:35:08.403833,modeling for 1,75,0.001350,0.002180,0.000831,modeling for 1
3,2019-07-25 06:35:17.778092,modeling for 4,89,0.001271,0.002003,0.000732,modeling for 4
4,2019-07-25 06:36:38.318599,modeling for 2,75,0.008555,0.009263,0.000709,modeling for 2
5,2019-07-25 06:37:22.798903,modeling for 6,79,0.019777,0.022784,0.003007,modeling for 6
6,2019-07-25 06:39:30.193157,modeling for 5,89,0.010091,0.010747,0.000656,modeling for 5
7,2019-07-25 06:39:40.228651,modeling for 7,64,0.001904,0.002756,0.000853,modeling for 7


In [11]:
print(mean_absolute_error(df_valid_pred.sort_values(by=['index']).reset_index(drop=True).predict.values, df_train.reset_index(drop=True).y.values))
for t in df_train.type.unique().tolist():
    index = df_train[df_train['type']==t]['index'].values
    print(t, mean_absolute_error(df_valid_pred[df_valid_pred['index'].isin(index)].sort_values(by=['index']).reset_index(drop=True).predict.values, df_train[df_train['index'].isin(index)].reset_index(drop=True).y.values))

0.0113439131210441
0 0.006486287590348556
3 0.019027581430896747
1 0.002180097990113484
4 0.0020032261513277276
2 0.009263426870801004
6 0.022783675627548072
5 0.010747013764359296
7 0.0027563504118763215


In [12]:
# df_test_pred = df_trial.loc[idx]['df_test_pred']
df_submit = pd.DataFrame()
df_submit['dso'] = np.mean(df_test_pred.drop(columns=['index']).values, axis=1)
df_submit['id'] = df_test_pred['index']
df_submit.to_pickle('../../data/oof/lgbm_dso_test.pkl')

In [13]:
df_submit = df_valid_pred.rename(columns={'predict':'dso', 'index':'id'})
df_submit.to_pickle('../../data/oof/lgbm_dso_train.pkl')